In [1]:
# https://medium.com/@niveditha.itengineer/learn-how-to-setup-portaudio-and-pyaudio-in-ubuntu-to-play-with-speech-recognition-8d2fff660e94

import sounddevice as sd
import numpy as np
import scipy.signal
import timeit
import python_speech_features
#import RPi.GPIO as GPIO

#from tflite_runtime.interpreter import Interpreter
import tensorflow as tf


2024-02-25 13:25:27.905965: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Parameters
#debug_time = 1
#debug_acc = 0
#led_pin = 8
#word_threshold = 0.5
rec_duration = 0.02
sample_rate = 16000

num_channels = 1
model_path = 'models3_30k/svdf/tflite_stream_state_external/stream_state_external.tflite'

# Sliding window
window = np.zeros(int(rec_duration * sample_rate) * 2)

# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# Load model (interpreter)
#interpreter = Interpreter(model_path)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

[{'name': 'input_audio', 'index': 0, 'shape': array([  1, 320], dtype=int32), 'shape_signature': array([  1, 320], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'streaming/speech_features/data_frame_1input_state', 'index': 1, 'shape': array([  1, 640], dtype=int32), 'shape_signature': array([  1, 640], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'streaming/svdf_0/depthwise_conv1d_6/depthwise_conv1d_6input_state', 'index': 2, 'shape': array([ 1,  4, 16], dtype=int32), 'shape_signature': array([ 1,  4, 16], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameter

In [3]:
inputs = []
for s in range(len(input_details)):
  inputs.append(np.zeros(input_details[s]['shape'], dtype=np.float32))

input_details[0]['shape']

array([  1, 320], dtype=int32)

In [4]:
# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    # Check to make sure we're downsampling
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    # We can only downsample by an integer factor
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))

    return resampled_signal, new_fs

In [11]:
# This gets called every 0.5 seconds
def sd_callback(rec, frames, time, status):   
    #print('rec:', rec.shape) 
    # Remove 2nd dimension from recording sample
    #rec = np.squeeze(rec)
    rec = np.reshape(rec, (1,-1))

    # set input audio data (by default input data at index 0)
    interpreter.set_tensor(input_details[0]['index'], rec)

    # set input states (index 1...)
    for s in range(1, len(input_details)):
        interpreter.set_tensor(input_details[s]['index'], inputs[s])

    # run inference
    interpreter.invoke()

    # get output: classification
    out_tflite = interpreter.get_tensor(output_details[0]['index'])
    #print(start / 16000.0, np.argmax(out_tflite), np.max(out_tflite))

    # get output states and set it back to input states
    # which will be fed in the next inference cycle
    for s in range(1, len(input_details)):
    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
        inputs[s] = interpreter.get_tensor(output_details[s]['index'])

    out_tflite_argmax = np.argmax(out_tflite)
    if out_tflite_argmax>1:
        print(out_tflite_argmax)


  

    


In [12]:
reset_state = True

# before processing new test sequence we can reset model state
# if we reset model state then it is not real streaming mode
if reset_state:
  for s in range(len(input_details)):
    print(input_details[s]['shape'])
    inputs[s] = np.zeros(input_details[s]['shape'], dtype=np.float32)

[  1 320]
[  1 640]
[ 1  4 16]
[ 1 10 32]
[ 1 10 32]
[ 1 10 32]
[ 1 10 64]
[  1  10 128]
[  1   1 128]


In [13]:
# Start streaming from microphone
print('channels:',num_channels, 'samplerate:',sample_rate, 'blocksize:', int(sample_rate * rec_duration))
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        pass

channels: 1 samplerate: 16000 blocksize: 320
13
13
12
18
18
18
12
12
12
12
12
4
18
13
20
13
20
20
32
19
19
19
19
19
19
19
19
19
19
19
19
19
10
19
19
32
32
32


KeyboardInterrupt: 